In [1]:
!pip install --quiet langchain langchain-openai langchain-community yfinance pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [21]:
# Step 3: Import Necessary Modules
from langchain.agents import AgentType, initialize_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
import yfinance as yf
import pandas as pd

In [22]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [23]:
# Step 4: Define a Custom Tool for Fetching Stock Data
def fetch_stock_data(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period='1mo', interval='1d')
        if hist.empty:
            return f"No historical data found for ticker symbol: {ticker}"
        # Convert the DataFrame to a string for display
        return hist.tail().to_string()
    except Exception as e:
        return f"An error occurred while fetching data for {ticker}: {str(e)}"

In [24]:
stock_data_tool = Tool(
    name="StockDataFetcher",
    func=fetch_stock_data,
    description="Fetches the last month's daily stock data for a given ticker symbol."
)

In [25]:
# Step 5: Initialize the Language Model and Agent with Tools
llm = ChatOpenAI(temperature=0.0)
tools = [YahooFinanceNewsTool(), stock_data_tool]

In [26]:
agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [29]:
# Step 6: Use the Agent to Process a Query
query = "What happened today with Tata Steel stocks?"
response = agent_chain.run(query)
print(response)



> Entering new AgentExecutor chain...
I should check the financial news for Tata Steel to see if there are any recent updates.
Action: yahoo_finance_news
Action Input: TATASTEEL.BO
Observation: No news found for company that searched with TATASTEEL.BO ticker.
Thought:There doesn't seem to be any recent news about Tata Steel, so I should check the stock data instead.
Action: StockDataFetcher
Action Input: TATASTEEL.BO
Observation:                                  Open        High         Low       Close   Volume  Dividends  Stock Splits
Date                                                                                                       
2025-04-21 00:00:00+05:30  137.949997  140.250000  137.000000  139.199997  1727408        0.0           0.0
2025-04-22 00:00:00+05:30  142.100006  142.850006  137.800003  138.100006  2241957        0.0           0.0
2025-04-23 00:00:00+05:30  139.250000  141.399994  137.300003  141.100006  1229528        0.0           0.0
2025-04-24 00:00:00+05:3